import numpy as np
import scipy as sp

In [ ]:
from quadrotor.dynamics import QuadrotorDynamicsBase, QuadrotorState
from quadrotor.controller import ControllerBase, QuadrotorCommands
from quadrotor.trajectory import TrajectoryBase, TrajectoryState

In [ ]:
from sym import Rot3

def dRot3(R: Rot3, omega: np.ndarray) -> Rot3:
    """
    NOTE(roy): If interested, we can discuss the math here later. There are other
    ways of doing this but to keep the ode_int formulation, we specifically require
    dq/dt. Refer to 'Baseile Graf - Quaternions and Dynamics' or (for intuition)
    https://math.stackexchange.com/questions/1896379/how-to-use-the-quaternion-derivative
    """
    quat = np.array(R.to_storage())  # Convert to raw quaternion
    (q0, q1, q2, q3) = quat  # xyzw storage, see https://symforce.org/api-gen-cpp/class/classsym_1_1Rot3.html

    G = np.array([[ q3,  q2, -q1, -q0],
                [-q2,  q3,  q0, -q1],
                [ q1, -q0,  q3, -q2]])
    quat_dot = (G.T @ omega) / 2
    return Rot3.from_storage(quat_dot)

In [ ]:
from dataclasses import dataclass, field

@dataclass
class Params:
    """A little utility class to hold our quadrotor parameters"""
    mass: float = 1.352  # [kg]

    # NOTE(roy) this is a matrix now as discussed!
    # NOTE(roy) don't get scared by the field/default_factory thing! It's just a way to do default field initialization for dataclasses,
    # see https://docs.python.org/3/library/dataclasses.html. Don't worry about it!
    inertia: np.ndarray = field(default_factory=lambda: np.array([
        [9.8e-3, 0, 0],
        [0, 10.02e-3, 0],
        [0, 0, 18.6e-3],
    ]))

    rotor_diameter: float = 10 * 0.0254  # [m] 10 inches

    static_thrust_coefficient: float = 0.14553  # [-] Thrust coefficient
    static_torque_coefficient: float = 0.01047  # [-] Torque coefficient

    # NOTE(roy) assume this is the same front-to-back and left-to-right
    arm_length: float = 0.3814 / 2.0  # [m] Arm length

    # Universal constants
    g: float = 9.80665  # [m/s2] Gravity
    rho: float = 1.225  # [kg/m3] Sea-level density

    def rotor_model(self, static_coefficient: float) -> float:
        # NOTE(roy) I'm just including this for completeness sake, this means we can e.g. see how air density
        # influences our simulation
        # Refer to this source for derivation: https://web.mit.edu/16.unified/www/FALL/thermodynamics/notes/node86.html
        return self.rho * static_coefficient * (self.rotor_diameter ** 4) / (4 * np.pi ** 2)

    # NOTE(roy) if you're unsure what @property decorators do: they make it so that we can call k_thrust as if it was just a class
    # variable. I.e. we'd write params.k_thrust now, rather than params.k_thrust()
    @property
    def k_thrust(self):
        return self.rotor_model(self.static_thrust_coefficient)

    @property
    def k_torque(self):
        return self.rotor_model(self.static_torque_coefficient)

# Simply treat as a global
p = Params()

### Writing a controller for our drone
In this notebook, we'll start writing a feedback controller for our model! Again, I will be leaving you guys alone a little bit more.

#### Recommended steps
- Copy your 3D quadrotor model from the previous lecture
- Work your way down in the `Controller` class
    - I implemented a basic structure, but feel free to diverge from this if you like!
    - Write the inverse mixing matrix formulation to from **u** to rotor rates
    - As discussed, the first week I expect you to just set up control in *z* to warm up!

### The dynamics

In [ ]:
# Copy your 3D Quadrotor here!

### The controller

In [ ]:
@dataclass
class ControllerParams:
    """The same as above but for the controller gains"""

    # NOTE(roy) diag just means we get a matrix with these values on the diagonal,
    # so these are 3x3 matrices (print them if you're unsure!)
    # We use this formulation because it makes the math a bit more readable later on!
    K_p: np.ndarray = field(default_factory=lambda: np.diag([0, 0, 100]))
    K_d: np.ndarray = field(default_factory=lambda: np.diag([0, 0, 20]))

    # Min/max rotor rates!
    rotor_rate_min: float = 180
    rotor_rate_max: float = 600

controller_p = ControllerParams()

In [ ]:
class Controller(ControllerBase):
    @staticmethod
    def rotor_rates_from_u(u: np.ndarray) -> np.ndarray:
        # NOTE(roy) implement your conversion from u (so that's [u1, u2_1, u2_2, u2_3] like we discussed, a 4-sized vector)
        # to rotor rates (which is also a four-sized vector)

        # NOTE(roy) the steps here are:
        #  1. convert u -> forces with the inverse mixing matrix
        #  2. convert forces -> rotor rates by inverting the rotor rate model
        #  3. limit the rotor rates with rotor_rate_min/rotor_rate_max above!
        raise NotImplementedError("rotor_rates_from_u not implemented!")

    def step(self, t: float, trajectory: TrajectoryState, state: QuadrotorState) -> QuadrotorCommands:
        desired_position, desired_velocity = trajectory.position, trajectory.velocity
        actual_position, actual_velocity = state.position, state.velocity
        
        # NOTE(roy) first compute the desired acceleration using the equation from the lectures!
        # I already extracted the values you need to compute it above
        commanded_acceleration = np.zeros(3)  # Your equation here!

        # NOTE(roy) now compute u_1 from the commanded acceleration using the linearized equation from the lectures!
        u_1 = 0  # Your equation here!

        # NOTE(roy) we're just doing thrust this week, set u_2 to just be zeros!
        u_2 = np.zeros(3)
        u = np.array([u_1, *u_2])

        rotor_rates = Controller.rotor_rates_from_u(u)
        return QuadrotorCommands(rotor_rates)

### The trajectory
Because we're now finally doing closed-loop control, we also need to set desired states in the trajectory!
I've set up something simple, please mess around with it!

In [ ]:
class JumpTrajectory(TrajectoryBase):
    def eval(self, t: float):
        altitude = 0 if t < 1.0 else 1.0
        position = np.array([0, 0, altitude])

        return TrajectoryState(t, position=position)

### Running it!

In [ ]:
from quadrotor.simulator import SimulatorBase, SimulatorState

# This might be slow to run the first time!
from quadrotor.renderer import animate_k3d, animate_matplotlib

In [ ]:
def run_and_render(initial_state: QuadrotorState, t_total: int = 2.0):
    sim = SimulatorBase(
        dt=0.01,
        dynamics=FullQuadrotorDynamics(),
        controller=Controller(),
        trajectory=JumpTrajectory(),
        initial_state=initial_state,
        t_total=t_total,
    )
    
    # Run the simulator
    output = sim.simulate()

    # K3D renderer
    plot = animate_k3d(output)
    
    plot.display()
    plot.start_auto_play()

In [ ]:
# NOTE(roy) play with this variable and see what happens!
initial_state = QuadrotorState(
    position=np.zeros(3),
    velocity=np.array([0, 0, 0]),
    orientation=Rot3.from_yaw_pitch_roll(0, 0, 0),
    angular_velocity=np.zeros(3),
)

run_and_render(initial_state, 5.0)

### Looking at the controller behavior
If all is well, the quadrotor should have a stable hover and after 1 second jump up 1 meter (with the `JumpTrajectory`)

Try some experiments:
- What happens if you increase/descrease `K_p`?
- What happens if you increase/decrease `K_d`?
- What happens if you use a slightly different mass in the controller (so the controller and dynamics have a different estimate for the quadrotor mass)? How do you think you could solve the behavior you see?

### Commit your updated notebook to your fork!